In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!git clone https://github.com/Tony607/object_detection_demo
%cd object_detection_demo/
!pip3 install -r requirements.txt

Cloning into 'object_detection_demo'...
remote: Enumerating objects: 124, done.
remote: Total 124 (delta 0), reused 0 (delta 0), pack-reused 124
Receiving objects: 100% (124/124), 11.16 MiB | 24.73 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/content/object_detection_demo/object_detection_demo


In [9]:
!ls

data			    README.md
deploy			    requirements.txt
generate_tfrecord.py	    resize_images.py
LICENSE			    tensorflow_object_detection_training_colab.ipynb
local_inference_test.ipynb  test
local_inference_test.py     xml_to_csv.py
